# **Telco Customer Churn Prediction**


## Project Content

<a id = 0></a>


### First Step: First Organization

1. [Introduction](#1)
2. [Loading libraries and packages to embark our new journey](#2)
3. [Loading and Checking The Dataset](#3)

### Second Step: Data Preprocessing

4. [Exploratory Data Analysis](#4)
5. [Numeric Fields Analysis](#5)
6. [Categorical Fields Analysis](#6)
7. [Feature Scaling](#7)
8. [Correlation Analysis](#8)
9. [Dealing with Outliers](#9)
10. [Determining Distributions of Numeric Fields](#10)
11. [Appllying One Hot Encoding to Categorical Fields](#11)
12. [Feature Scaling with The RobustScaler Method](#12)
13. [Seperating Data into Two Parts of Train and Test](#13)

### Third and Final Step: Modeling

16. [Modelling (Logistic Regression)](#16)
17. [Cross Validation (Logistic Regression)](#17)
18. [AUC-ROC Curve (Logistic Regression)](#18)
19. [Hyper Parameter Optimization (Logistic Regression)](#19)
20. [Modelling (Decision Tree)](#20)
21. [Cross Validation (Decision Tree)](#21)
22. [AUC-ROC Curve (Decision Tree)](#22)
23. [Hyper Parameter Optimization (Decision Tree)](#23)
24. [Modelling (Support Vector Classifier)](#24)
25. [Cross Validation (Support Vector Classifier)](#25)
26. [AUC-ROC Curve (Support Vector Classifier)](#26)
27. [Hyper Parameter Optimization (Support Vector Classifier)](#27)
28. [Modelling (Random Forest)](#28)
29. [Cross Validation (Support Vector Classifier)](#29)
30. [AUC-ROC Curve (Support Vector Classifier)](#30)
31. [Hyper Parameter Optimization (Support Vector Classifier)](#31)


## 1. Introduction <a id = 1></a>


Text


[Project Content](#0)


## 2. Loading libraries and packages to embark our new journey <a id = 2></a>


In [43]:
# Basic Python Packages

import warnings
warnings.filterwarnings("ignore")

# Numpy Library

import numpy as np

# Pandas Library and Settings

import pandas as pd
pd.set_option("max_columns", None)
pd.options.display.max_colwidth = 100

# Visualization Libraries (Matplotlib, Seaborn, Missingno)

import matplotlib.pyplot as plt
import seaborn as sns
import missingno

%matplotlib inline

In [44]:
def f_num_unique_values(dataframe, fields = None):
    """
    By this function we can check how many unique values each field has

    Args:
        dataframe (pandas.core.frame.DataFrame): The main dataframe
        fields (list): The fields we want to check number of their unique values
    """
    
    num_unique_values_df = pd.DataFrame(columns = ["Field_Name", "Unique_Values_Num"])

    if fields == None:
        fields = dataframe.columns
        
    num_unique_values = []

    for field in fields:
        
        num_unique_values += [dataframe[field].nunique()]
        
    num_unique_values_df["Field_Name"] = fields
    num_unique_values_df["Unique_Values_Num"] = num_unique_values
        
    return num_unique_values_df

In [45]:
def f_unique_values(dataframe, fields):
    """
    By this function we can get a dataframe and categorical fields and show a dataframe of unique values of each field.

    Args:
        dataframe (pandas.core.frame.DataFrame): The main dataframe
        fields (list): The fields we want to check their unique values
        
    Returns:
        unique_values_df (pandas.core.frame.DataFrame): A dataframe of fields and their unique values
    """
    
    unique_values_df = pd.DataFrame(columns = ["Field_Name", "Unique_Values"])

    fields_list = []
    unique_values_list = []

    for field in fields:

        fields_list += [field]
        field_unique_values_list = list(dataframe[field].unique())
            
        unique_values_str = ""
            
        for values_str in field_unique_values_list:
            if field_unique_values_list.index(values_str) != len(field_unique_values_list) - 1:
                unique_values_str += str(values_str) + " | "
            else:
                unique_values_str += str(values_str)                 
            
        unique_values_list += [unique_values_str]
        
    unique_values_df["Field_Name"] = fields_list
    unique_values_df["Unique_Values"] = unique_values_list
        
    return unique_values_df

In [46]:
def f_cat_to_int(dataframe, fields = None):
    """
    This function helps to change categorical fields' values to numerical format.

    Args:
        dataframe (pandas.core.frame.DataFrame): The main dataframe
        fields (list): The fields which we want to duplicate a integer format of them

    Returns:
        dataframe (pandas.core.frame.DataFrame): A dataframe which categorical fields have integer values
    """
    
    if fields == None:
        fields = list(dataframe.columns)
        
    cat_dict = {}

    for field in fields:
        
        cat_dict[field + "_str"] = list(dataframe[field].unique())
        cat_dict[field + "_int"] = [i for i in range(0, len(list(dataframe[field].unique())))]

        dataframe[field + "_int"] = dataframe[field]
        dataframe[field + "_int"].replace(cat_dict[field + "_str"], cat_dict[field + "_int"], inplace = True)
        
    return dataframe

In [47]:
def f_sorted_correlations(dataframe, fields = None, omited_fields = None, target_field = None, cat_num = None):
    """
    This function get a dataframe and fields to create a dataframe of correlation based on our demands.
    It can show a dataframe of correlations between each two fields in a sorted manner and also it can show a one row sorted correlation values between non-target fields and the target.

    Args:
        dataframe (pandas.core.frame.DataFrame): The main dataframe
        fields (list, optional): The fields we want to check their correlation
        omited_fields (list, optional): The fields we dont want to check their correlation
        target_field (list, optional): The target field we want to check its correlation with other fields
        cat_num (list, optional): The target field we want to check its correlation with other fields
        
    Returns:
        corr_df (pandas.core.frame.DataFrame): Final correlation matrix
    """
    
    if fields == None:
        fields = list(dataframe.columns)
    
    corr_dict = {}
    all_correlation_df = dataframe[fields].corr()
    
    if cat_num == None:
        
        if omited_fields != None:
            all_correlation_df.drop(omited_fields, axis = 0, inplace = True)
            all_correlation_df.drop(omited_fields, axis = 1, inplace = True)
        
        if target_field == None:
            for row in range(0, len(list(all_correlation_df.columns))):
                for column in range(row, len(list(all_correlation_df.columns))):
                    if row < column:
                        corr_dict[f"{list(all_correlation_df.columns)[row]}, {list(all_correlation_df.columns)[column]}"] = all_correlation_df.iloc[row, column]

            corr_df = pd.DataFrame(columns = ["Fields", "Correlation_Value"])

            corr_df["Fields"] = list(corr_dict.keys())
            corr_df["Correlation_Value"] = list(corr_dict.values())
            corr_df.sort_values(by = "Correlation_Value", inplace = True)
            
        else:
            corr_df = all_correlation_df[[target_field]]
            corr_df = corr_df.sort_values(by = target_field)
            corr_df.drop(target_field, axis = 0, inplace = True)
            corr_df = corr_df.T
            
    else:
        
        corr_df = ((dataframe[cat_num[0] + cat_num[1]].corr()[cat_num[0]]).T)[cat_num[1]]

    return corr_df

In [48]:
def f_vif_function(dataframe, omited_fields = None):
    """
    This function show a dataframe with sorted values of VIF of each field.

    Args:
        dataframe (pandas.core.frame.DataFrame): The main dataframe
        omited_fields (list, optional): The fields we dont want to check their correlation
                
    Returns:
        vif_info (pandas.core.frame.DataFrame): _description_
    """
    
    if omited_fields != None:
        dataframe.drop(omited_fields, axis = 1, inplace = True)
        
    vif_dict = {}
    
    vif_dict["VIF"] = [variance_inflation_factor(dataframe.values, i) for i in range(dataframe.shape[1])]
    vif_df = ((pd.DataFrame(vif_dict, index = dataframe.columns)).sort_values(by = "VIF").T)
    
    return vif_df

In [49]:
def f_train_dev_split(dataframe, target_field):
    """
    This function gets a dataframe and split it into two train and development dataframes.

    Args:
        dataframe (pandas.core.frame.DataFrame): The main dataframe
        target_field (sring): The field we want to create the linear regression model based on that

    Returns:
        x_train, x_develop, y_train, y_develop (pandas.core.frame.DataFrame): Anything we want for modeling
    """
    
    x = dataframe.drop([target_field], axis = 1)
    y = dataframe[target_field]
    
    x_train, x_develop, y_train, y_develop = train_test_split(x, y, test_size = 0.20, random_state = 3)

    print(f"Train (x) dataset shape is:      {x_train.shape}")
    print(f"Train (y) dataset shape is:      {y_train.shape}")

    print(f"Dev (x) dataset shape is:       {x_develop.shape}")
    print(f"Dev (y) dataset shape is:       {y_develop.shape}")
    
    return x_train, x_develop, y_train, y_develop

In [50]:
def f_linear_regression(train , dev):
    """
    This function has been defined to model a linear regression and show R squared value of train and development datasets.

    Args:
        train (list): A list of both x and y dataframes for training the model
        dev (list): A list of both x and y dataframes for checking the model

    Returns:
        results (pandas.core.frame.DataFrame): R squared value of the model of both train and dev datasets
    """
    
    model = LinearRegression()
    model.fit(train[0], train[1])
    
    results = pd.DataFrame(columns = ["Train", "Dev"], index = ["R Squared"])
    results["Train"] = [model.score(train[0], train[1])]
    results["Dev"] = [model.score(dev[0], dev[1])]
    
    fields = list((train[0]).columns)
    coefficients = list(model.coef_)
    coefficients_dict = {fields[i]: coefficients[i] for i in range(len(fields))}
    sorted_coefficients_dict = sorted(coefficients_dict.items(), key = lambda x : x[1])
    
    intercept = model.intercept_
    
    print(f"Coefficients values:")
    
    for i in range(len(fields)):
        print(f"    * {sorted_coefficients_dict[i][0]} : {round((sorted_coefficients_dict[i][1]), 2)}")
    
    print(f"\nInetrcept: {model.intercept_}\n")
    
    return results

[Project Content](#0)


## 3. Loading and Checking The Dataset <a id = 3></a>


### Loading The Dataset


In [51]:
df = pd.read_csv("Telco-Customer-Churn.csv")

### Checking The Dataset


In [52]:
df.head(5)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


### Omiting Redundant Fields


In [53]:
df.drop(["customerID"], axis = 1, inplace = True)

### Mending The Datset Fields' Names


In [54]:
fields = [field.capitalize() for field in list(df.columns)]

df.columns = fields

### Examining Missing Values


In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Gender            7043 non-null   object 
 1   Seniorcitizen     7043 non-null   int64  
 2   Partner           7043 non-null   object 
 3   Dependents        7043 non-null   object 
 4   Tenure            7043 non-null   int64  
 5   Phoneservice      7043 non-null   object 
 6   Multiplelines     7043 non-null   object 
 7   Internetservice   7043 non-null   object 
 8   Onlinesecurity    7043 non-null   object 
 9   Onlinebackup      7043 non-null   object 
 10  Deviceprotection  7043 non-null   object 
 11  Techsupport       7043 non-null   object 
 12  Streamingtv       7043 non-null   object 
 13  Streamingmovies   7043 non-null   object 
 14  Contract          7043 non-null   object 
 15  Paperlessbilling  7043 non-null   object 
 16  Paymentmethod     7043 non-null   object 


### Examining Unique Values


In [56]:
f_num_unique_values(df)

,Field_Name,Unique_Values_Num
0,Gender,2
1,Seniorcitizen,2
2,Partner,2
3,Dependents,2
4,Tenure,73
5,Phoneservice,2
6,Multiplelines,3
7,Internetservice,3
8,Onlinesecurity,3
9,Onlinebackup,3


### Separating Fields to Numerical and Categorical

In [57]:
num_fields = ["Tenure", "Monthlycharges", "Totalcharges"]

cat_fields = list(df.columns)

for field in num_fields:
    cat_fields.remove(field) 

In [58]:
print("Categorical fields are:")

f_unique_values(df, fields = cat_fields)

Categorical fields are:


,Field_Name,Unique_Values
0,Gender,Female | Male
1,Seniorcitizen,0 | 1
2,Partner,Yes | No
3,Dependents,No | Yes
4,Phoneservice,No | Yes
5,Multiplelines,No phone service | No | Yes
6,Internetservice,DSL | Fiber optic | No
7,Onlinesecurity,No | Yes | No internet service
8,Onlinebackup,Yes | No | No internet service
9,Deviceprotection,No | Yes | No internet service


In [59]:
print("Numerical fields are:")

(df[num_fields].describe()).T

Numerical fields are:


,count,mean,std,min,25%,50%,75%,max
Tenure,7043.0,32.371149,24.559481,0.00,9.0,29.00,55.00,72.00
Monthlycharges,7043.0,64.761692,30.090047,18.25,35.5,70.35,89.85,118.75


[Project Content](#0)


## 4. Exploratory Data Analysis <a id = 4></a>


### Examining Statistics of Numeric Fields


### Examining Statistics of Categorical Fields


### Examining The Missing Data


### Filling The Missing Values


[Project Content](#0)


## 5. Numeric Fields Analysis <a id = 5></a>


### Bi-Variate Analysis between Numeric Fields and Target


### Analysis between Numeric Fields Among Themselves


### Correlation Analysis between Numeric Fields and Target


[Project Content](#0)


## 6. Categorical Fields Analysis <a id = 6></a>


### Bi-Variate Analysis between Categorical Fields and Target


### Correlation Analysis between Categorical Fields and Target


[Project Content](#0)


## 7. Feature Scaling <a id = 7></a>


[Project Content](#0)


## 8. Correlation Analysis <a id = 8></a>


### Numeric fields and Target Value Correlation Analysis


### Numeric and Categorical Fields Correlation Analysis


### Dropping Columns with Low Correlation


[Project Content](#0)


## 9. Dealing with Outliers <a id = 9></a>


[Project Content](#0)


## 12. Determining Distributions of Numeric Fields <a id = 12></a>


[Project Content](#0)


## 13. Appllying One Hot Encoding to Categorical Fields <a id = 13></a>


[Project Content](#0)


## 14. Feature Scaling with The RobustScaler Method <a id = 14></a>


[Project Content](#0)


## 15. Seperating Data into Two Parts of Train and Test <a id = 15></a>


[Project Content](#0)


## 16. Modelling (Logistic Regression) <a id = 16></a>


## 17. Cross Validation (Logistic Regression) <a id = 17></a>


## 18. AUC-ROC Curve (Logistic Regression) <a id = 18></a>


## 19. Hyper Parameter Optimization (Logistic Regression) <a id = 19></a>


## 20. Modelling (Desicion Tree) <a id = 20></a>


## 21. Cross Validation (Desicion Tree) <a id = 21></a>


## 22. AUC-ROC Curve (Desicion Tree) <a id = 22></a>


## 23. Hyper Parameter Optimization (Desicion Tree) <a id = 23></a>


## 24. Modelling (Support Vector Classification) <a id = 24></a>


### 25. Cross Validation (Support Vector Classification) <a id = 25></a>


## 26. AUC-ROC Curve (Support Vector Classification) <a id = 26></a>


## 27. Hyper Parameter Optimization (Support Vector Classification) <a id = 27></a>


## 28. Modelling (Random Forest Classification) <a id = 28></a>


## 29. Cross Validation (Random Forest Classification) <a id = 29></a>


## 30. AUC-ROC Curve (Random Forest Classification) <a id = 30></a>


## 31. Hyper Parameter Optimization (Random Forest Classification) <a id = 31></a>
